In [131]:
import numpy as np

npzfile = np.load('codon_hmm_excl1.npz')
outcomes = npzfile['outcomes']
outcomes_dict = {'CAT': 21, 'TTC': 65, 'GAT': 38, 'ATC': 14, 'TGT': 63, 'AAT': 4, 'AGA': 9, 'ATG': 15, 'TAC': 53, 'AAA': 1, 'ACG': 7, 'GAA': 35, 'GTG': 49, 'TGA': 60, 'CTC': 31, 'GAC': 36, 'GTT': 50, 'GGA': 43, 'ATT': 16, 'TGG': 62, 'A': 0, 'GTC': 48, 'TTT': 67, 'CGC': 27, 'TCA': 56, 'AAC': 2, 'CTA': 30, 'GCG': 41, 'CGA': 26, 'GCC': 40, 'TCT': 59, 'AGT': 12, 'TTA': 64, 'CCT': 25, 'GAG': 37, 'TCG': 58, 'C': 17, 'TAA': 52, 'T': 51, 'ATA': 13, 'ACC': 6, 'ACA': 5, 'CCC': 23, 'TAT': 55, 'CAA': 18, 'AGG': 11, 'TTG': 66, 'CAG': 20, 'G': 34, 'AAG': 3, 'TAG': 54, 'ACT': 8, 'CAC': 19, 'GTA': 47, 'CTG': 32, 'CGG': 28, 'GCT': 42, 'AGC': 10, 'GGT': 46, 'GCA': 39, 'CCG': 24, 'CCA': 22, 'GGG': 45, 'CGT': 29, 'GGC': 44, 'TGC': 61, 'CTT': 33, 'TCC': 57}
state_codes = npzfile['state_codes']


def read_fasta_file(filename):
    """
    Reads the given FASTA file f and returns a dictionary of sequences.

    Lines starting with ';' in the FASTA file are ignored.
    """
    sequences_lines = {}
    current_sequence_lines = None
    with open(filename) as fp:
        for line in fp:
            line = line.strip()
            if line.startswith(';') or not line:
                continue
            if line.startswith('>'):
                sequence_name = line.lstrip('>')
                current_sequence_lines = []
                sequences_lines[sequence_name] = current_sequence_lines
            else:
                if current_sequence_lines is not None:
                    current_sequence_lines.append(line)
    sequences = {}
    for name, lines in sequences_lines.items():
        sequences[name] = ''.join(lines)
    return sequences

def write_fasta_file(annotation_file_to_save, annotation_string, genome_file=None):
    if (genome_file!=None):
        with open(genome_file) as gf:
            first_line = gf.readline()
            s = " "
            words = first_line.split(s)
            words[0] = words[0] + " gene annotation"
            first_line = s.join(words[:-1])
    else:
        first_line = "; gene annotation"
    with open(annotation_file_to_save, 'w+') as af:
        af.write(first_line + '\n')
        name = annotation_file_to_save.replace('/', '\\').split('\\')[-1].split('.fa')[0]
        af.write('>' + name + '\n')
        line_length = 60
        full_lines = len(annotation_string)//line_length
        for anot_line in range(full_lines):
            af.write(annotation_string[anot_line*line_length:(anot_line+1)*line_length] + '\n')
        af.write(annotation_string[(full_lines)*line_length:])
        

def alphabet(l):
    if type(l) is str:
        if l == "":
            return -1
        else:
            return outcomes_dict[l]
    else:
        return [l2i(l_) for l_ in l]

    
def i2l(i):
    if type(i) is int:
        return outcomes[i]
    else:
        return [outcomes[i_] for i_ in i]
    
def emit(string_to_match, maxsize=3):
    prob = np.log(np.zeros([E_.shape[0], maxsize+1]))
    length = np.zeros([E_.shape[0]])
    emit_prob = np.log(np.zeros([E_.shape[0]]))
    match = np.zeros([E_.shape[0]])
    if len(string_to_match) >= maxsize:
        for state in range(E_.shape[0]):
            for l in range(maxsize+1):
                try:
                    prob[state,l] = E_[state, alphabet( string_to_match[:l] ) ]
                except:
                    prob[state,l] = np.log(0)
            length[state] = np.argmax(prob[state,:])
            emit_prob[state] = prob[state,length[state]]
    return emit_prob, length



def viterbi_gen(A, E, p, s):
    p = p.reshape([-1])
    result = -1*np.ones([A.shape[0], len(s)])
    result[:, 0] = p
    I = np.log(np.eye(A.shape[0]))
    q = np.int_(np.zeros(p.shape[0])) # not probabilities! so no log here.
    for i in range(1, len(s)):
        if i % 1000 == 0:
            print("\r" + str(i), end="")
            
        # freeze transistions FROM a frozen state to other states (only keep the diagonal)
        freeze = (q>0).reshape([-1])
        
        Atemp = np.copy(A)
        Atemp[freeze,:] = I[freeze,:] 
        a = (Atemp.T + result[:, i-1]).max(axis=1)
        
        horizon = np.minimum(3,len(s[i:]))
        e, l = emit(s[i:i+horizon], horizon)
        l = l.reshape([-1])
        b = a + e
        
        # b skal være max(a+e, prev b)
        b[freeze] = np.maximum(b[freeze], result[freeze, i-1])
        q[np.invert(freeze)] = l[np.invert(freeze)]
        q += -1
        
        #print( s[i:i+np.minimum(3,len(s[i:]))], '\na', a, '\ne:', e, '\nb:', b, '\nq:', q, '\nfreeze:', freeze)

        result[:, i] = b
    return result

def viterbi_path(X, A, E, s):
    n = X.shape[1]-1
    path = np.zeros(n+1, dtype=np.int32)
    path[n] = np.argmax(X[:, n], axis=0)
    for i in range(n-1, -1, -1):
        p = E[path[i+1], alphabet(s[i+1])]
        p = p + X[:, i]
        p = p + A[:, path[i+1]]
        path[i] = np.argmax(p)
    return path

def viterbi(A, E, p, s):
    X = viterbi_gen(A, E, p, s)
    return viterbi_path(X, A, E, s)


In [115]:
#
# compare_anns.py <true> <pred>
#
# compares a predicted gene structure against the true gene structure and computes
# various statistics summarizing the quality of the prediction. The argument <true> is 
# the true gene structure in faste format, and <pred> is the predicted gene structure 
# in fasta format, e.g.
#
# > python compare_anns.py ./annotation1.fa ./pred1.fa
# > Only Cs (tp=728238, fp=0, tn=505177, fn=249):
# > Sn = 0.9997, Sp = 1.0000, CC = 0.9996, AC = 0.9996
# > Only Rs (tp=618777, fp=0, tn=505426, fn=0):
# > Sn = 1.0000, Sp = 1.0000, CC = 1.0000, AC = 1.0000
# > Both (tp=1347015, fp=0, tn=505177, fn=249):
# > Sn = 0.9998, Sp = 1.0000, CC = 0.9997, AC = 0.9997
#
# Christian Storm <cstorm@birc.au.dk>
#

import os
import sys
import string
import math

def read_ann(filename):
    lines = []
    for l in open(filename).readlines():
        if l[0] != ">" and l[0] != ';':
            lines.append(l.strip())
    return "".join(lines)

def count_c(true, pred):
    total = tp = fp = tn = fn = 0
    for i in range(len(true)):
        if pred[i] == 'C' or pred[i] == 'c':
            total = total + 1
            if true[i] == 'C' or true[i] == 'c':
                tp = tp + 1
            else:
                fp = fp + 1
        if pred[i] == 'N' or pred[i] == 'n':
            if true[i] == 'N' or true[i] == 'n' or true[i] == 'R' or true[i] == 'r':
                tn = tn + 1
            else:
                fn = fn + 1
    return(total, tp, fp, tn, fn)

def count_r(true, pred):
    total = tp = fp = tn = fn = 0
    for i in range(len(true)):
        if pred[i] == 'R' or pred[i] == 'r':
            total = total + 1
            if true[i] == 'R' or true[i] == 'r':
                tp = tp + 1
            else:
                fp = fp + 1
        if pred[i] == 'N' or pred[i] == 'n':
            if true[i] == 'N' or true[i] == 'n' or true[i] == 'C' or true[i] == 'c':
                tn = tn + 1
            else:
                fn = fn + 1
    return(total, tp, fp, tn, fn)

def count_cr(true, pred):
    total = tp = fp = tn = fn = 0
    for i in range(len(true)):
        if pred[i] == 'C' or pred[i] == 'c' or pred[i] == 'R' or pred[i] == 'r':
            total = total + 1
            if (pred[i] == 'C' or pred[i] == 'c') and (true[i] == 'C' or true[i] == 'c'):
                tp = tp + 1
            elif (pred[i] == 'R' or pred[i] == 'r') and (true[i] == 'R' or true[i] == 'r'):
                tp = tp + 1                
            else:
                fp = fp + 1
        if pred[i] == 'N' or pred[i] == 'n':
            if true[i] == 'N' or true[i] == 'n':
                tn = tn + 1
            else:
                fn = fn + 1
    return(total, tp, fp, tn, fn)

def print_stats(tp, fp, tn, fn):
    sn = float(tp) / (tp + fn)
    sp = float(tp) / (tp + fp)
    cc = float((tp*tn - fp*fn)) / math.sqrt(float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    acp = 0.25 * (float(tp)/(tp+fn) + float(tp)/(tp+fp) + float(tn)/(tn+fp) + float(tn)/(tn+fn))
    ac = (acp - 0.5) * 2
    print("Sn = %.4f, Sp = %.4f, CC = %.4f, AC = %.4f" % (sn, sp, cc, ac))

def print_all(true, pred):
    print(count_c(true, pred))
    (totalc, tp, fp, tn, fn) = count_c(true, pred)
    totalc += 1
    if totalc > 0:
        print("Only Cs (tp=%d, fp=%d, tn=%d, fn=%d):" % (tp, fp, tn, fn))
        print_stats(tp, fp, tn, fn)
    (totalr, tp, fp, tn, fn) = count_r(true, pred)
    totalr += 1
    if totalr > 0:
        print("Only Rs (tp=%d, fp=%d, tn=%d, fn=%d):" % (tp, fp, tn, fn))
        print_stats(tp, fp, tn, fn)

    (total, tp, fp, tn, fn) = count_cr(true, pred)
    if totalc > 0 and totalr > 0:
        print("Both (tp=%d, fp=%d, tn=%d, fn=%d):" % (tp, fp, tn, fn))
        print_stats(tp, fp, tn, fn)
    
    
def compare_anns(true, pred):
    # Read true annotation
    true_ann = read_ann(true)

    # Read predicted annotations
    pred_ann = read_ann(pred)

    # Check annoation length
    error = 0
    if len(true_ann) != len(pred_ann):
        print("ERROR: The lengths of two predictions are different")
        print("Expected %d, but found %d" % (len(true_ann), len(pred_ann)))
        sys.exit(1) 

    # Print stats
    print_all(true_ann, pred_ann)


In [130]:


for i in range(1,2):
    print('\ngenome%d' % i)
    npzfile = np.load('codon_hmm_excl%d.npz' % i)
    A_ = np.log(npzfile['A'])
    E_ = np.log(npzfile['E'])
    p_ = np.log(npzfile['pi'])
    genome = read_fasta_file('data/genome%d.fa' % i)['genome%d' % i][:100]
    
    gen = viterbi_gen(A_, E_, p_, genome)
    print(gen)
    path = viterbi_path(gen, A_, E_, p_, genome)
    print(path)
    string = ""
    #for r in res:
    #    string += state_codes[r]
    print(np.sum(res[res==1]))
    print(np.sum(res[res==4]))
    
    #write_fasta_file('data/pred%d.fa' % i, string, 'data/genome%d.fa' % i)
    #compare_anns('data/annotation%d.fa' % i, 'data/pred%d.fa' % i)
    
    


genome1
[[   0.           -1.08499418   -2.91154663   -3.99654081   -5.08153499
    -6.90808744   -7.99704365   -9.08203783  -10.17099405  -11.25598823
   -12.34098241  -14.16660546  -15.25159964  -17.07815209  -18.16314627
   -19.24814045  -20.33313463  -21.41812881  -22.50312299  -23.58811717
   -25.41374023  -26.49873441  -27.58372859  -28.66872277  -29.75371695
   -30.83871113  -31.92370531  -33.01266152  -11.12388873  -12.20888291
   -13.29387709  -14.37887127  -15.46386545  -17.28948851  -19.11511156
   -20.20406778  -22.02969083  -23.11864705  -24.20364123  -26.03019367
   -27.11914989   -8.15974671   -9.24870293  -10.33765915  -11.42661536
   -12.51160954  -13.60056576  -15.42711821   -6.29056608   -7.37556026
    -9.20211271  -10.29106893   -6.0350941    -7.12405031   -8.21300653
   -10.03862958  -11.1275858   -12.21654201  -13.30153619  -14.39049241
   -16.21704486   -7.42439569   -9.25094814  -11.07750059  -12.16249477
   -13.98904722  -15.0740414   -16.89966445  -18.725287

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: RuntimeWarning: divide by zero encountered in log
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: RuntimeWarning: divide by zero encountered in log
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: RuntimeWarning: divide by zero encountered in log
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:93: RuntimeWarning: divide by zero encountered in log
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:72: RuntimeWarning: divide by zero encountered in log
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:74: RuntimeWarning: divide by zero encountered in log
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:82: RuntimeWarning: divide by zero encountered in log
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:84: VisibleDeprecationWarning: using a non-integer number instead of an integer will resu

In [116]:
compare_anns('data/annotation1.fa', 'data/pred1.fa')

(0, 0, 0, 1123954, 728487)
Only Cs (tp=0, fp=0, tn=1123954, fn=728487):


ZeroDivisionError: float division by zero

In [96]:
# compare to noncoding
for i in range(1,2):
    print('\ngenome%d' % i)
    genome = read_fasta_file('data/genome%d.fa' % i)['genome%d' % i]
    
    string = state_codes[np.zeros(len(genome))]
    
    write_fasta_file('data/pred%d.fa' % i, string, 'data/genome%d.fa' % i)
    compare_anns('data/annotation%d.fa' % i, 'data/pred%d.fa' % i)
    


genome1


IndexError: arrays used as indices must be of integer (or boolean) type